# P1 Edgar data client for REST API

## Initialization

In [9]:
%env P1_EDGAR_API_URL=https://data.particle.one/edgar/v2/
%env P1_EDGAR_API_TOKEN=8c9c9458b145202c7a6b6cceaabd82023e957a46d6cf7061ed8e1c94a168f2fd

env: P1_EDGAR_API_URL=https://data.particle.one/edgar/v2/
env: P1_EDGAR_API_TOKEN=8c9c9458b145202c7a6b6cceaabd82023e957a46d6cf7061ed8e1c94a168f2fd


In [10]:
import os

import p1_data_client_python.edgar_client as p1_edg

# Enter your token here.
# You can get your token by signing up at `www.particle.one`.
#TOKEN = "YOUR_TOKEN_HERE"
# An example token is like:
P1_API_URL = os.environ["P1_EDGAR_API_URL"]
print("P1_API_URL=", P1_API_URL)

P1_API_TOKEN = os.environ["P1_EDGAR_API_TOKEN"]
print("P1_API_TOKEN=", P1_API_TOKEN)

P1_API_URL= https://data.particle.one/edgar/v2/
P1_API_TOKEN= 8c9c9458b145202c7a6b6cceaabd82023e957a46d6cf7061ed8e1c94a168f2fd


## Quick start

There are 3 steps: 
1. Get information about ticker identifiers 
2. Get information about financial items available
3. Download data

## Mappers

### GvkCikMapper

It handles CIK <-> GVK transformation.

In [11]:
gvk_mapper = p1_edg.GvkCikMapper(base_url=P1_API_URL, token=P1_API_TOKEN)
gvk_mapper.get_gvk_from_cik(cik='0000940800', as_of_date='2007-01-18')

,cik,effdate,thrudate,gvk
0,0000940800,2007-01-18T00:00:00,2007-03-14T23:59:59,061411


In [12]:
gvk_mapper.get_cik_from_gvk(gvk='061411', as_of_date='2007-01-18')

,cik,effdate,thrudate,gvk
0,0000940800,2007-01-18T00:00:00,2007-03-14T23:59:59,061411


### ItemMapper

It provides mapping between keywords and description of Compustat items.

In [13]:
item_mapper = p1_edg.ItemMapper(base_url=P1_API_URL, token=P1_API_TOKEN)
item_mapper.get_item_from_keywords(
            keywords=['short-term', 'short term'])

,item,description
0,IVSTCHY,Short-Term Investments - Change
1,CHEQ,Cash and Short-Term Investments
2,ALTOQ,Other Long-term Assets
3,IVLTQ,Total Long-term Investments
4,DLTTQ,Long-Term Debt - Total
5,DLTRY,Long-Term Debt - Reduction
6,DLTISY,Long-Term Debt - Issuance


In [14]:
item_mapper.get_mapping()

,item,description
0,ACCHGQ,Accounting Changes / Cumulative Effect
1,ACOMINCQ,Accumulated Other Comprehensive Income (Loss)
2,ACOQ,Current Assets - Other - Total
3,ACTQ,Current Assets - Total
4,ADRRQ,ADR Ratio
...,...,...
167,XIDOQ,Extraordinary Items and Discontinued Operations
168,XINTQ,Interest and Related Expense- Total
169,XIQ,Extraordinary Items
170,XRDQ,Research and Development Expense


## Payload data

After collecting all necessary identifiers we are ready to download the payload data.
- form_name: Edgar form short code
- cik: Company Identification Key. Optional. Also, could be a list of cik. 
- start_date and end_date: limit our selection by dates
- items: list of items that we want to see in a result set.

Keep in mind that if you specify a broad range of search, the result could be very large.

In [17]:
client = p1_edg.EdgarClient(base_url=P1_API_URL, token=P1_API_TOKEN)

client.get_payload(
            form_name="form8k",
            cik=18498,
            start_date="2020-01-04",
            end_date="2020-12-04",
            item="ACT_QUARTER",
        )

,form_uuid,filing_url,form_publication_timestamp,filing_date,creation_timestamp,cik,ticker,item_name,form_table_row_name,item_value,compustat_timestamp,period_of_report,compustat_coifnd_id,gvk
0,514be1a9-88d1-426b-be7e-d7b5a0ff64e6,https://www.sec.gov/Archives/edgar/data/18498/...,2020-03-12T11:45:58,2020-03-12,2020-06-12 23:49:03.580,18498,GCO,ACT_QUARTER,Total current assets,508.18300000000005,2020-03-12T20:00:00,2020-01-31T00:00:00,10099821,5109
1,30cc5db5-e9ce-4d43-830b-d5d5a94b201a,https://www.sec.gov/Archives/edgar/data/18498/...,2020-09-03T11:30:54,2020-09-03,2020-09-03 11:32:58.186,18498,GCO,ACT_QUARTER,Total current assets,777.658,None,2020-07-31T00:00:00,,5109
2,70a8c7b9-d0a7-4c41-902c-89d84d3b9122,https://www.sec.gov/Archives/edgar/data/18498/...,2020-06-09T11:21:05,2020-06-09,2020-12-01 17:13:22.396,18498,GCO,ACT_QUARTER,Total current assets,735.008,2020-06-09T20:00:00,2020-04-30T00:00:00,10164135,5109


In [18]:
client.get_payload(
            form_name="form8k",
            cik=[18498, 319201, 5768]
        )

,form_uuid,filing_url,form_publication_timestamp,filing_date,creation_timestamp,cik,ticker,item_name,form_table_row_name,item_value,compustat_timestamp,period_of_report,compustat_coifnd_id,gvk
0,c5584895-6261-4449-9845-1ffae0acdc3b,https://www.sec.gov/Archives/edgar/data/18498/...,2016-12-02T12:09:31,2016-12-02,2020-06-12 21:12:47.669,18498,GCO,ACO_QUARTER,Other current assets,88.969,2016-12-02T21:00:00,2016-10-31T00:00:00,9121134,5109
1,c5584895-6261-4449-9845-1ffae0acdc3b,https://www.sec.gov/Archives/edgar/data/18498/...,2016-12-02T12:09:31,2016-12-02,2020-06-12 21:12:47.669,18498,GCO,ACT_QUARTER,Total current assets,894.573,2016-12-02T21:00:00,2016-10-31T00:00:00,9121134,5109
2,c5584895-6261-4449-9845-1ffae0acdc3b,https://www.sec.gov/Archives/edgar/data/18498/...,2016-12-02T12:09:31,2016-12-02,2020-06-12 21:12:47.669,18498,GCO,AP_QUARTER,Accounts payable,247.282,2016-12-02T21:00:00,2016-10-31T00:00:00,9121134,5109
3,c5584895-6261-4449-9845-1ffae0acdc3b,https://www.sec.gov/Archives/edgar/data/18498/...,2016-12-02T12:09:31,2016-12-02,2020-06-12 21:12:47.669,18498,GCO,CHE_QUARTER,Cash and cash equivalents,30.52,2016-12-02T21:00:00,2016-10-31T00:00:00,9121134,5109
4,c5584895-6261-4449-9845-1ffae0acdc3b,https://www.sec.gov/Archives/edgar/data/18498/...,2016-12-02T12:09:31,2016-12-02,2020-06-12 21:12:47.669,18498,GCO,DLC_QUARTER,Current portion long-term debt,12.172,2016-12-02T21:00:00,2016-10-31T00:00:00,9121134,5109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,60a92f86-bfa8-4adc-b8c5-0b517ad51571,https://www.sec.gov/Archives/edgar/data/5768/0...,2010-08-09T20:13:08,2010-08-09,2020-06-13 01:15:52.318,5768,ASEI,SALE_QUARTER,Total net sales and contract revenues,53.643,2010-08-10T04:00:00,2010-06-30T00:00:00,7153084,1554
912,b0afa528-1457-4476-bbae-bfac01adf31f,https://www.sec.gov/Archives/edgar/data/5768/0...,2015-11-09T21:10:49,2015-11-09,2020-06-13 01:23:40.678,5768,ASEI,SALE_QUARTER,Total net sales and contract revenues,25.136999999999997,2015-11-10T05:00:00,2015-09-30T00:00:00,8733305,1554
913,a350f89d-cfaa-4d52-9514-5f146c91070d,https://www.sec.gov/Archives/edgar/data/5768/0...,2011-02-08T21:08:41,2011-02-08,2020-06-13 01:34:29.280,5768,ASEI,AO_QUARTER,Other assets,5.432,None,2010-12-31T00:00:00,7263012,1554
914,d303ca47-ab39-4c50-9129-6c8163f5e176,https://www.sec.gov/Archives/edgar/data/5768/0...,2011-11-07T21:10:52,2011-11-07,2020-06-13 01:41:54.087,5768,ASEI,NI_QUARTER,Net income,6.851,2011-11-08T05:00:00,2011-09-30T00:00:00,7509872,1554
